In [3]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

import pickle
import json

In [4]:
df = pd.read_csv("tmdb_5000_movies.csv")

In [5]:
df.head(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [6]:
df.shape

(4803, 20)

In [7]:
df.loc[0,"genres"]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [8]:
df.loc[1, "genres"]

'[{"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 28, "name": "Action"}]'

In [10]:
sum(df["overview"].isna())

3

In [11]:
df["overview"] = df["overview"].fillna("")

# Also for Using Genres and Keywords data.

In [12]:
df["genres"]

,genres
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."
...,...
4798,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam..."
4799,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""..."
4800,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam..."
4801,[]


In [13]:
df["keywords"]

,keywords
0,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."
2,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name..."
3,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,..."
4,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":..."
...,...
4798,"[{""id"": 5616, ""name"": ""united states\u2013mexi..."
4799,[]
4800,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam..."
4801,[]


In [14]:
print(df["genres"].isna().sum())
print(df["keywords"].isna().sum())

0
0


## Trying for Correct Matching

In [15]:
keywords = json.loads(df["keywords"][0])
kah = [j["name"] for j in keywords]
kah = " ".join(kah)

In [16]:
genres = json.loads(df["genres"][0])
ah = [j["name"] for j in genres]
bah = " ".join(ah)

In [17]:
kah + " " + bah

'culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d Action Adventure Fantasy Science Fiction'

## Configuring Dataframe Mapper

In [18]:
def apply_genres_keywords(row):
    keywords = json.loads(row["keywords"])
    keywords_string = [j["name"] for j in keywords]
    keywords_string = " ".join(keywords_string)

    genres = json.loads(row["genres"])
    genres_string = [j["name"] for j in genres]
    genres_string = " ".join(genres_string)
    overview_string = row["overview"]

    return keywords_string + " " + genres_string + " " + overview_string

In [19]:
df["query"] = df.apply(apply_genres_keywords, axis=1)

In [20]:
df.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,query
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,culture clash future space war space colony so...
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,ocean drug abuse exotic island east india trad...
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,spy based on novel secret agent sequel mi6 bri...


In [21]:
vectorizer = TfidfVectorizer(
    stop_words="english",
    min_df=2,
    max_df=.95,
)

tfidf_matrix = vectorizer.fit_transform(df["query"])

In [22]:
feature_names = vectorizer.get_feature_names_out()
print("Feature names:")
print(feature_names)
print("\nlength of feature names:")
print(len(feature_names))

Feature names:
['00' '000' '007' ... 'zoologists' 'zoom' 'zorro']

length of feature names:
12268


## For Simplicity Create a Series for title indexer

In [23]:
title_index = pd.Series(df.index, index=df["title"])

In [24]:
title_index

,0
title,
Avatar,0
Pirates of the Caribbean: At World's End,1
Spectre,2
The Dark Knight Rises,3
John Carter,4
...,...
El Mariachi,4798
Newlyweds,4799
"Signed, Sealed, Delivered",4800


In [29]:
def most_similar_movies(movie, many=5):
    try:
        idx = title_index[movie]
    except KeyError:
        print("Movie not found")
        return
    query_vector = tfidf_matrix[idx]
    scores = cosine_similarity(query_vector, tfidf_matrix).flatten()
    idxs =  (- scores).argsort()[1: many+1]
    scores = scores[idxs]

    return df.iloc[idxs]["title"]

most_similar_movies("The Dark Knight", 8)

,title
3,The Dark Knight Rises
428,Batman Returns
299,Batman Forever
119,Batman Begins
1359,Batman
3854,"Batman: The Dark Knight Returns, Part 2"
9,Batman v Superman: Dawn of Justice
210,Batman & Robin


In [30]:
most_similar_movies("Inception", 4)

,title
2897,Cypher
4401,The Helix... Loaded
1803,Blood and Wine
1267,Duplex


In [31]:
most_similar_movies("Interstellar", 4)

,title
373,Mission to Mars
2966,2001: A Space Odyssey
1531,Moonraker
4332,Silent Running


In [32]:
most_similar_movies("Oppenheimer", 4)

Movie not found


In [34]:
most_similar_movies("Ford vs Ferrari", 4)

Movie not found


In [35]:
most_similar_movies("Rush", 4)

,title
405,The Fast and the Furious: Tokyo Drift
3100,The Brown Bunny
225,Speed Racer
560,Driven


# Thank God!

In [36]:
most_similar_movies("Avengers", 4)

Movie not found
